In [4]:
!pip install opencv-python



In [5]:
!pip install tensorflow

In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model


In [7]:
model = load_model("emotion_cnn_model.h5")


In [8]:
emotion_labels = {
    0: "angry",
    1: "disgust",
    2: "fear",
    3: "happy",
    4: "sad",
    5: "surprise",
    6: "neutral"
}

learning_state_map = {
    "angry": "Frustrated – Needs support",
    "disgust": "Disengaged – Needs attention",
    "fear": "Confused – Needs clarification",
    "sad": "Low motivation – Needs encouragement",
    "happy": "Engaged – Learning well",
    "surprise": "Curious – Actively learning",
    "neutral": "Normal – Average engagement"
}


In [9]:
face_cascade = cv2.CascadeClassifier(
    "haarcascade_frontalface_default.xml"
)

if face_cascade.empty():
    print("Face cascade NOT loaded")
else:
    print("Face cascade loaded successfully")


Face cascade loaded successfully


In [10]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Webcam not accessible")
else:
    print("Webcam opened successfully")


Webcam opened successfully


In [11]:
cap = cv2.VideoCapture(0)

In [12]:
while True:
    ret, frame = cap.read()

    if not ret:
        print("Camera not working")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = face / 255.0
        face = face.reshape(1, 48, 48, 1)
        

        pred = model.predict(face, verbose=0)
        emotion = emotion_labels[np.argmax(pred)]
        learning_state = learning_state_map[emotion]
        cv2.imshow("Face sent to CNN", face.reshape(48,48))
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(
            frame,
            f"{emotion} | {learning_state}",
            (x, y-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0,255,0),
            2
        )

    cv2.imshow("Emotion Based Learning Detector", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [17]:
cap.release()
cv2.destroyAllWindows()
